# Mini-proyecto: Algoritmo FP-Tree
- Ortega Ibarra Jaime Jesus.

**Objetivo del mini-proyecto:** 

Implementar el código del algoritmo FP-Tree en una libreta usando python.
Requisitos:
- La identificación de los elementos frecuentes deberá realizarse con al menos dos valores de soporte diferentes describiendo qué observaste y si se repitió algún conjunto de elementos.
- El código debe estar debidamente documentado y funcionando.
- Para la identificación de los elementos frecuentes deberás usar la siguiente base de datos: Groceries dataset (https://www.kaggle.com/heeraldedhia/groceries-dataset?select=Groceries_dataset.csv)

# Exploración de los datos

Mediante la librería $Pandas$ realizamos la lectura de los datos para poder visualizarlos.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Groceries_dataset.csv')

In [3]:
data.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
len(data)

38765

In [5]:
len(data['itemDescription'].value_counts())

167

- Contamos con 38765 Registros
- Los datos se dividen en:
    + Member_number: Número de la membresía con la cual se realizó la compra
    + Date: Fecha de la compra
    + itemDescription: Producto que se compró

### Transformando a transacciones

Dado que se cuenta con la fecha de la compra, así como el número de la membresía, podemos ordenar los diferentes artículos, de tal manera que puedan expresar la compra completa.

In [6]:
fechas = list(set(data[data['Member_number'] == 3180]["Date"]))

In [7]:
fechas

['04-05-2015',
 '09-11-2014',
 '15-09-2015',
 '06-02-2015',
 '30-07-2014',
 '24-02-2014',
 '02-11-2014',
 '19-10-2015',
 '03-07-2015',
 '15-03-2015']

In [8]:
Membresias = list(set(data["Member_number"]))

In [9]:
def transacciones(data_frame, membresia,lista):
    '''
    En esta función se realizará un recorrido por todo el conjunto de datos,
    de tal manera que se puedan expresar los diferentes productos como
    una transacción de compra.
    
    Data_frame: Conjunto de datos
    Membresia: Número de membresía dentro del conjunto de datos
    lista: Lista de python en la cual se almacenarán los datos
    '''
    fechas = list(set(data[data['Member_number'] == membresia]["Date"]))
    data_cliente = data[data['Member_number'] == membresia]
    for i in range(len(fechas)):
        freq_temp = []
        transaccion = data_cliente[data_cliente['Date'] == fechas[i]]
        items = list(transaccion['itemDescription'])
        freq_temp.append(items)
        freq.append(freq_temp)

In [10]:
def quitar_corchetes(text):
    '''
    Recibe los diferentes datos de tipo String y realiza una limpieza para 
    eliminar carácteres no deseados.
    
    text: Cadena de texto
    '''
    acentos = (("[", " "),
        ("]", " "))
    for i, k in acentos:
        text = text.replace(i, k).replace(i.upper(), k.upper())
    return text

In [11]:
freq = []
for i in range(len(Membresias)):
    transacciones(data, Membresias[i], freq)

In [12]:
Transacciones = pd.DataFrame(freq)
Transacciones.columns = ['Transacciones']

In [13]:
for i in range(len(Transacciones)):
    Transacciones['Transacciones'][i] = (quitar_corchetes(str(Transacciones['Transacciones'][i])))

Como podemos observar en la siguiente tabla, encontramos los artículos que se han comprado por cada diferente membresía en cada fecha diferente.

In [14]:
Transacciones.head()

,Transacciones
0,"'sausage', 'hygiene articles'"
1,"'canned beer', 'misc. beverages'"
2,"'whole milk', 'pastry', 'salty snack'"
3,"'soda', 'pickled vegetables'"
4,"'sausage', 'whole milk', 'semi-finished bread..."


In [15]:
total_items = ''
for i in range(len(Transacciones)):
    total_items += str(Transacciones['Transacciones'][i]).replace("\n", ",")

In [16]:
def normalizar(text):
    '''
    Función que nos ayudará a normalizar nuestros artículos
    
    text: Cadena de texto
    '''
    text = text.replace("[^a-zA-Z#]", " ")
    text = text.lower()
    acentos = (
        ("."," "),
        (","," "),
        ("”", " "),
        ("“", " "),
        ("'", " ")
    )
    for i, k in acentos:
        text = text.replace(i, k).replace(i.upper(), k.upper())
    return text

In [17]:
text1 = normalizar(total_items)
text1 = text1.split("    ")

In [18]:
len(text1)

38765

## Obtenemos los productos más frecuentes

Para este punto decidimos realizar mediante dos diferentes soportes $0.03$ y $0.05$

## Frecuencia

In [19]:
def frecuencias(data):
    '''
    Esta función recibe un conjunto de datos, dado esto obtiene la cantidad de ocasiones 
    que se repiten los diferentes items (Frecuencia).
    
    Data: Conjunto de datos'''
    frecuencias = []
    valores = list(set(data["itemDescription"]))
    for k in range(len(valores)):
        aux = 0
        for i in range(len(data)):
            if data["itemDescription"][i] == valores[k]:
                aux += 1
        frecuencias.append(aux)
    return frecuencias

In [20]:
frecuencias_values = frecuencias(data)

In [21]:
len(frecuencias_values)

167

Almacenamos nuestras frecuencias en una lista y a su vez las añadimos dentro de un $DataFrame$ junto con los valores únicos dentro de los items, los cuales obtenemos mediante la función $set()$

In [22]:
Frecuencia = pd.DataFrame(set(data["itemDescription"]))
Frecuencia["Frecuencia"] = frecuencias_values

In [23]:
Frecuencia.columns = ["Elementos", "Frecuencia"]

In [24]:
Frecuencia.head(10)

,Elementos,Frecuencia
0,skin care,20
1,ice cream,227
2,pasta,121
3,margarine,491
4,female sanitary products,40
5,dishes,135
6,nut snack,22
7,house keeping products,45
8,cookware,17
9,instant coffee,61


## Frecuencia Sup = 0.03

Para obtener el soporte únicamente dividimos la frecuencia de cada uno de nuestros items, entre el total de transacciones.

In [25]:
soportes = []
for i in range(len(frecuencias_values)):
    soportes.append(frecuencias_values[i]/len(Transacciones))
Frecuencia["Soporte"] = soportes

### Filtramos nuestros datos con un soporte mínimo de 0.03 y ordenamos de acuerdo a su frecuencia

In [26]:
Freq_minsup3 = Frecuencia[Frecuencia["Soporte"] >= 0.03]
Freq_minsup3 = Freq_minsup3.sort_values(by = "Frecuencia", ascending=False)
Freq_minsup3

,Elementos,Frecuencia,Soporte
68,whole milk,2502,0.167212
133,other vegetables,1898,0.126846
23,rolls/buns,1716,0.114683
144,soda,1514,0.101183
56,yogurt,1334,0.089153
134,root vegetables,1071,0.071577
165,tropical fruit,1032,0.068970
114,bottled water,933,0.062354
98,sausage,924,0.061752
123,citrus fruit,812,0.054267


In [27]:
print("Elementos con soporte mínimo = 0.03: ", len(Freq_minsup3))

Elementos con soporte mínimo = 0.03:  27


Con el siguiente ciclo, vamos a recorrer cada una de nuestras transacciones, comparando con nuestros items más frecuentes obtenidos en la tabla anterior, ordenados de manera descendente, en caso de que estos sean iguales serán almacenado dentro de una lista, la cual será almacenada dentro de otra lista y de esta manera obtendremos una lista final con las transacciones con los productos más frecuentes.

In [28]:
trans_list = []
for k in range(len(Transacciones)):
    trans_frec = []
    for j in range(len(Freq_minsup3)): 
        for i in range(len(Transacciones['Transacciones'][k].replace("'", "").split(", "))):
            Transaccions = Transacciones['Transacciones'][k].replace("'", "")

            if Transaccions.split(", ")[i].replace(" ", "") == Freq_minsup3['Elementos'].values[j].replace(" ", ""):
                items = str(Freq_minsup3['Elementos'].values[j])
                trans_frec.append(items)

    trans_list.append(trans_frec)

Eliminamos las listas vacías, pues nos inidican transacciones en las cuales no llevaron ningún artículo dentro de nuestra lista de artículos frecuentes.

In [29]:
Transacciones_final = []
for i in range(len(trans_list)):
    if trans_list[i] != []:
        Transacciones_final.append(trans_list[i])

In [30]:
print('Número de transacciones frecuentes con Soporte Mínimo de 0.03: ',len(Transacciones_final))

Número de transacciones frecuentes con Soporte Mínimo de 0.03:  13351


## Lista de transacciones con artículos más frecuentes.

In [31]:
Transacciones_final

[['sausage'],
 ['canned beer'],
 ['whole milk', 'pastry'],
 ['soda'],
 ['whole milk', 'yogurt', 'sausage'],
 ['whole milk', 'soda'],
 ['soda', 'whipped/sour cream', 'frankfurter'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['frankfurter', 'curd'],
 ['beef'],
 ['other vegetables'],
 ['tropical fruit'],
 ['whole milk', 'butter'],
 ['root vegetables'],
 ['rolls/buns', 'sausage'],
 ['rolls/buns', 'rolls/buns'],
 ['rolls/buns'],
 ['other vegetables', 'shopping bags'],
 ['whole milk', 'root vegetables', 'pastry'],
 ['other vegetables'],
 ['whole milk', 'tropical fruit', 'pip fruit'],
 ['whole milk', 'rolls/buns'],
 ['canned beer'],
 ['whipped/sour cream', 'margarine'],
 ['rolls/buns', 'rolls/buns'],
 ['whole milk'],
 ['shopping bags'],
 ['whole milk', 'rolls/buns', 'bottled beer', 'frankfurter', 'frankfurter'],
 ['rolls/buns', 'bottled water'],
 ['soda',
  'yogurt',
  'yogurt',
  'root vegetables',
  'tropical fruit',
  'domestic eggs'],
 ['yogurt', 'newspapers'],
 ['yogurt', 'tropical frui

## Frecuencia Sup = 0.05

Repetimos todo el proceso anterior, pero en esta ocasión con un soporte mínimo de 0.05

In [32]:
Freq_minsup5 = Frecuencia[Frecuencia["Soporte"] >= 0.05]
Freq_minsup5 = Freq_minsup5.sort_values(by = "Frecuencia", ascending=False)
Freq_minsup5

,Elementos,Frecuencia,Soporte
68,whole milk,2502,0.167212
133,other vegetables,1898,0.126846
23,rolls/buns,1716,0.114683
144,soda,1514,0.101183
56,yogurt,1334,0.089153
134,root vegetables,1071,0.071577
165,tropical fruit,1032,0.068970
114,bottled water,933,0.062354
98,sausage,924,0.061752
123,citrus fruit,812,0.054267


In [33]:
print("Elementos con soporte mínimo = 0.05: ", len(Freq_minsup5))

Elementos con soporte mínimo = 0.05:  11


In [34]:
trans_list = []
for k in range(len(Transacciones)):
    trans_frec = []
    for j in range(len(Freq_minsup5)): 
        for i in range(len(Transacciones['Transacciones'][k].replace("'", "").split(", "))):
            Transaccions = Transacciones['Transacciones'][k].replace("'", "")
            if Transaccions.split(", ")[i].replace(" ", "") == Freq_minsup5['Elementos'].values[j].replace(" ", ""):
                items = str(Freq_minsup5['Elementos'].values[j])
                trans_frec.append(items)

    trans_list.append(list(trans_frec))

In [35]:
Transacciones_final = []
for i in range(len(trans_list)):
    if trans_list[i] != []:
        Transacciones_final.append(trans_list[i])

In [36]:
print('Número de transacciones frecuentes con Soporte Mínimo de 0.03: ',len(Transacciones_final))

Número de transacciones frecuentes con Soporte Mínimo de 0.03:  10174


### Obtenemos nuestras transacciones de productos frecuentes dentro de nuestro soporte mínimo

In [37]:
Transacciones_final

[['sausage'],
 ['whole milk', 'pastry'],
 ['soda'],
 ['whole milk', 'yogurt', 'sausage'],
 ['whole milk', 'soda'],
 ['soda'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['other vegetables'],
 ['tropical fruit'],
 ['whole milk'],
 ['root vegetables'],
 ['rolls/buns', 'sausage'],
 ['rolls/buns', 'rolls/buns'],
 ['rolls/buns'],
 ['other vegetables'],
 ['whole milk', 'root vegetables', 'pastry'],
 ['other vegetables'],
 ['whole milk', 'tropical fruit'],
 ['whole milk', 'rolls/buns'],
 ['rolls/buns', 'rolls/buns'],
 ['whole milk'],
 ['whole milk', 'rolls/buns'],
 ['rolls/buns', 'bottled water'],
 ['soda', 'yogurt', 'yogurt', 'root vegetables', 'tropical fruit'],
 ['yogurt'],
 ['yogurt', 'tropical fruit'],
 ['pastry'],
 ['rolls/buns'],
 ['bottled water'],
 ['bottled water'],
 ['whole milk', 'other vegetables', 'yogurt'],
 ['pastry'],
 ['rolls/buns', 'bottled water', 'citrus fruit'],
 ['yogurt'],
 ['rolls/buns', 'root vegetables', 'tropical fruit'],
 ['whole milk'],
 ['whole milk'],
 ['bottle

## Conclusiones

- El procesamiento de los datos fue un tanto complicado al ser texto con el cual estaba trabajando, pues al realizar las transacciones de acuerdo a la membresía utilizada y la fecha registrada, no se almacenaban de la manera adecuada por diversos carácteres provocados por espacios y saltos de línea, por lo cual decidí remover dichos carácteres.

- Dentro de los resultados, como era de esperarse, mediante un soporte mínimo más alto, nuestra cantidad de elementos frecuentes disminuye, esto a su vez nos disminuye el total de transacciones frecuentes, pues con soporte mínimo 0.03 obtenemos 13351 transacciones, mientras con un soporte mínimo de 0.05 obtenemos 10174 transacciones.

- Entre los pares más comunes podemos encontrar Leche entera / Yogurh, así como Leche entera / Rolls-buns, lo cual me pareció bastante obvio, pues quien no desea su leche con algún pan.

- De igual manera, si vemos los items de manera individual, el item con mayor frecuencia es la Leche entera, seguido de Vegetales, mientras en los items con menor frecuencia tenemos los productos para cocina y cosméticos para bebé.
